# 최종 7월 앙상블 코드입니다.

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1. 코드를 실행하기 전에 아래 test_obs_path에 test data의 실제 기상 관측 데이터 경로를 입력하여주세요.
## 그래야 최종 sAError값이 출력됩니다.
# 2. xgboost_july.ipynb로 생성된 xgboost_output.csv와 mlp_july.ipynb로 생성된 mlp_output.csv의 경로가 하단의 xgb_pred와 mlp_pred의 경로와 일치하나 필히 확인 부탁드립니다. 
# 3. (선택) 필요시 최종 예측 결과 csv 파일인 final_submission.csv 의 경로도 수정하세요.


In [11]:
###############################################################
# test 채점용 관측데이터 경로
test_obs_path="./obs_dummy.csv"
###############################################################

###############################################################
# 최종 제출 csv (7월 관측 데이터에 대한 예측)
final_submission_path="./final_submission.csv"
###############################################################


In [ ]:
############# xgboost 단일 모델 7월 예측 결과 csv 파일의 경로
xgb_pred = pd.read_csv('./xgboost_output.csv')
############ forecastMLP 단일 모델 7월 예측 결과 csv 파일의 경로
mlp_pred = pd.read_csv('./mlp_output.csv')




obs = pd.read_csv(f'{test_obs_path}')

# 앙상블 실행

In [13]:
def compute_saerror(y_true_df, y_pred_df):
    rmse_temp = np.sqrt(mean_squared_error(y_true_df['temp'], y_pred_df['temp']))
    mae_temp  = mean_absolute_error(y_true_df['temp'], y_pred_df['temp'])
    sa_temp   = (rmse_temp + mae_temp) / 2

    rmse_hum = np.sqrt(mean_squared_error(y_true_df['humidity'], y_pred_df['humidity']))
    mae_hum  = mean_absolute_error(y_true_df['humidity'], y_pred_df['humidity'])
    sa_hum   = (rmse_hum + mae_hum) / 2

    rmse_pres = np.sqrt(mean_squared_error(y_true_df['pressure'], y_pred_df['pressure']))
    mae_pres  = mean_absolute_error(y_true_df['pressure'], y_pred_df['pressure'])
    sa_pres   = (rmse_pres + mae_pres) / 2

    total_score = sa_temp * 0.5 + sa_hum * 0.3 + sa_pres * 0.2
    return total_score, (sa_temp, sa_hum, sa_pres)

In [14]:
def apply_best_ensemble_and_save(mlp_df, xgb_df, true_df,
                                  temp_ratio=0.65, humidity_ratio=0.70, pressure_ratio=0.40):
    # 가중치 계산
    result_df = pd.DataFrame()
    result_df['temp']     = mlp_df['temp']     * temp_ratio     + xgb_df['temp']     * (1 - temp_ratio)
    result_df['humidity'] = mlp_df['humidity'] * humidity_ratio + xgb_df['humidity'] * (1 - humidity_ratio)
    result_df['pressure'] = mlp_df['pressure'] * pressure_ratio + xgb_df['pressure'] * (1 - pressure_ratio)

    # 성능 평가
    total_score, (sa_temp, sa_hum, sa_pres) = compute_saerror(true_df, result_df)

    print("앙상블 결과 저장 완료: ensemble_result.csv")
    print(f"Temp     sAError: {sa_temp:.4f}  (MLP {temp_ratio:.2f} / XGB {1-temp_ratio:.2f})")
    print(f"Humidity sAError: {sa_hum:.4f}  (MLP {humidity_ratio:.2f} / XGB {1-humidity_ratio:.2f})")
    print(f"Pressure sAError: {sa_pres:.4f}  (MLP {pressure_ratio:.2f} / XGB {1-pressure_ratio:.2f})")
    print(f"\nFinal Weighted sAError: {total_score:.4f}")

    return result_df, total_score, (sa_temp, sa_hum, sa_pres)

In [15]:
obs.rename(columns={'기상관측일시':'datetime',
                    '습도(%)':'humidity',
                    '기온(degC)':'temp',
                    '대기압(mmHg)':'pressure'}, inplace = True)
obs["datetime"] = pd.to_datetime(obs["datetime"])

In [16]:
mlp_pred["datetime"] = pd.to_datetime(mlp_pred["datetime"])
obs["datetime"] = pd.to_datetime(obs["datetime"])

# 공통 datetime만 남기기
obs_matched = obs[obs["datetime"].isin(mlp_pred["datetime"])].reset_index(drop=True)

In [17]:
result_df, total_score, sa_scores = apply_best_ensemble_and_save(
    mlp_pred, xgb_pred, obs,
    temp_ratio=0.45,
    humidity_ratio=0.6,
    pressure_ratio=0.40
)

앙상블 결과 저장 완료: ensemble_result.csv
Temp     sAError: 0.7611  (MLP 0.45 / XGB 0.55)
Humidity sAError: 5.2665  (MLP 0.60 / XGB 0.40)
Pressure sAError: 0.7593  (MLP 0.40 / XGB 0.60)

Final Weighted sAError: 2.1124


In [18]:
# 저장
result_df['datetime'] = obs_matched['datetime']
result_df.rename(columns={'datetime':'기상관측일시',
                    'humidity':'습도(%)',
                    'temp':'기온(degC)',
                    'pressure':'대기압(mmHg)'}, inplace = True)
result_df.to_csv(f"{final_submission_path}", index=False)